In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

from src.acnets.pipeline import Parcellation, ConnectivityExtractor, ConnectivityAggregator
from sklearn.preprocessing import LabelEncoder
from typing_extensions import deprecated


In [2]:
# Xy
subjects = Parcellation(atlas_name='difumo_64_2mm').fit_transform(None).coords['subject'].values
X = subjects.reshape(-1,1)                                  # subjects ids, shape: (n_subjects, 1)

y_encoder = LabelEncoder()
y = y_encoder.fit_transform([s[:4] for s in subjects])      # labels (AVGP=1 or NVGP=1), shape: (n_subjects,)


In [3]:
import numpy as np

p = Parcellation('dosenbach2010')
p_res = p.fit_transform(X)

c = ConnectivityExtractor('correlation')
c_res = c.fit_transform(p_res)

a = ConnectivityAggregator('network')
a_res = a.fit_transform(c_res)
a_res

<xarray.Dataset>
Dimensions:              (timepoint: 124, region: 160, subject: 32,
                          region_src: 160, region_dst: 160, network_src: 6,
                          network_dst: 6)
Coordinates:
  * timepoint            (timepoint) int64 1 2 3 4 5 6 ... 120 121 122 123 124
  * region               (region) <U19 'vmPFC 1' ... 'post occipital 160'
  * subject              (subject) <U9 'AVGP01' 'AVGP02' ... 'NVGP19NEW'
  * region_src           (region_src) <U19 'vmPFC 1' ... 'post occipital 160'
  * region_dst           (region_dst) <U19 'vmPFC 1' ... 'post occipital 160'
  * network_src          (network_src) object 'CER' 'CON' 'DMN' ... 'SMN' 'VIS'
  * network_dst          (network_dst) object 'CER' 'CON' 'DMN' ... 'SMN' 'VIS'
Data variables: (12/13)
    timeseries           (subject, timepoint, region) float64 259.0 ... 601.1
    age                  (subject) float64 ...
    sex                  (subject) <U1 ...
    group                (subject) <U4 ...
    preprocessed_beh     (subject) float64 ...
    preprocessed_rsfmri  (subject) float64 ...
    ...                   ...
    x                    (region) int64 ...
    y                    (region) int64 ...
    z                    (region) int64 ...
    index                (region) int64 ...
    network              (region) <U3 'DMN' 'FPN' 'FPN' ... 'VIS' 'VIS' 'VIS'
    connectivity         (subject, network_src, network_dst) float64 0.5036 ....

In [4]:
# Flatten connectivity matrix

@deprecated('Use full connectivity matrix instead')
def flatten(ds, mask_k=0):

    ds = ds.copy()
    import numpy as np
    node_type = ds['connectivity'].dims[-1][:-4]
    c = ds['connectivity'].values
    c[..., *np.tril_indices(c.shape[1], k=mask_k)] = np.nan
    ds['connectivity'].values = c
    c = ds['connectivity'].to_dataframe().dropna().reset_index().rename(columns={
        'connectivity': 'partial correlation'})
    c['connectivity'] = c[f'{node_type}_src'] + ' - ' + c[f'{node_type}_dst']
    c = c.pivot(index='subject', columns='connectivity', values='partial correlation')
    return c